## Tasks

### Precision change - done
### Image pre-processed size, 
### Number of max tokens, 
### Gradient accumulation

In [ ]:
from PIL import Image
import datasets
import numpy as np
import torch
import requests
import os

In [ ]:
from tqdm import tqdm

In [ ]:
from datasets import load_dataset
from datasets import Dataset

In [ ]:
from transformers import BlipProcessor, BlipForQuestionAnswering

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("ybelkada/blip-vqa-base", torch_dtype=torch.float16).to("cuda")

In [ ]:
ds_visdial = datasets.load_dataset('jxu124/visdial', split="train")

In [ ]:
ds_visdial

In [ ]:
ds_visdial = ds_visdial.remove_columns('global_image_id')
ds_visdial = ds_visdial.remove_columns('anns_id')

In [ ]:
ds_visdial

In [ ]:
img_list = ds_visdial['image_path']

In [ ]:
dialog = ds_visdial['dialog']

In [ ]:
capList = ds_visdial['caption']

In [ ]:
len(img_list), len(dialog)

In [ ]:
len(ds_visdial['image_path'])

In [ ]:
ds_visdial['image_path'][82770:]

In [ ]:
len(dialog), len(img_list)

In [ ]:
img_list

In [ ]:
questionList = []
ansList = []
img_path_list = []
captionList = []

for i in tqdm(range(123287)):
    
    dialogSet = dialog[i]
    img = img_list[i]
    caption = capList[i]
    
    for j in range(10):
        question = dialogSet[j][0]
        answer = dialogSet[j][1]
        
        questionList.append(question)
        ansList.append(answer)
        img_path_list.append(img)
        captionList.append(caption)


In [ ]:
listToDictionary = {'caption':captionList, 'questions':questionList, 'answers':ansList, 'images':img_path_list}
modified_train_set = Dataset.from_dict(listToDictionary)

In [ ]:
modified_train_set

In [ ]:
ds_visdial

In [ ]:
path_map = {"coco/val2014": "coco/train2014"}

In [ ]:

from dataclasses import dataclass

@dataclass
class ReplaceImagePath():
    path_map: {}
    def __call__(self, features):
        print(path_map)
        for k, v in self.path_map.items():
            features['images'] = features['images'].replace(k, v)
        return features
    
modified_train_set = modified_train_set.map(ReplaceImagePath(path_map=path_map)).cast_column("images", datasets.Image())


### Training  Set Ready Now

In [ ]:
modified_train_set = modified_train_set.shuffle(seed = 42)

In [ ]:
modified_train_set

### Processing Validation Set

In [ ]:
ds_validation_visdial = datasets.load_dataset('jxu124/visdial', split="validation")

In [ ]:
ds_validation_visdial

In [ ]:
ds_validation_visdial = ds_validation_visdial.remove_columns('global_image_id')
ds_validation_visdial = ds_validation_visdial.remove_columns('anns_id')

In [ ]:
ds_validation_visdial

In [ ]:
dialog = ds_validation_visdial['dialog']
img_list = ds_validation_visdial['image_path']
capList = ds_validation_visdial['caption']

In [ ]:
questionList_val = []
ansList_val = []
img_path_list_val = []
captionList_val = []

for i in tqdm(range(2064)):
    
    dialogSet = dialog[i]
    img = img_list[i]
    caption = capList[i]
    
    for j in range(10):
        question = dialogSet[j][0]
        answer = dialogSet[j][1]
        
        questionList_val.append(question)
        ansList_val.append(answer)
        img_path_list_val.append(img)
        captionList_val.append(caption)


In [ ]:
len(questionList_val), len(ansList_val), len(img_path_list_val)

In [ ]:
listToDictionary = {'caption':captionList_val, 'questions':questionList_val, 'answers':ansList_val, 'images':img_path_list_val}
modified_val_set = Dataset.from_dict(listToDictionary)

In [ ]:
modified_val_set

In [ ]:
modified_val_set = modified_val_set.cast_column("images", datasets.Image())

In [ ]:
modified_val_set

In [ ]:
shuffled_val_set = modified_val_set.shuffle(seed=42)
temp = shuffled_val_set[0:200]
small_val_set = Dataset.from_dict(temp)

In [ ]:
modified_val_set['questions']

##### End processing Validation Set

#### Finetuning Code Starts Here

In [ ]:
from torch.utils.data import DataLoader
from datasets import Dataset

In [ ]:
class visdial_dataset(Dataset):
    
    def __init__(self, dataset, processor):
        self.processor = processor
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self,idx):
        item = self.dataset[idx]
        
        encodings = self.processor(images = item["images"], text = item["questions"], padding = "max_length", return_tensors = "pt")
        labels = self.processor(text = item['answers'], padding = "max_length", return_tensors = "pt").input_ids

        encodings['labels'] = labels
        encodings = {k:v.squeeze() for k,v in encodings.items()}

        return encodings

In [ ]:
train_dataset_object = visdial_dataset(modified_train_set, processor)
train_dataloader = DataLoader(train_dataset_object, shuffle=True, batch_size=16)

In [ ]:
batch = next(iter(train_dataloader))

for k,v in batch.items():
  print(k, v.shape)

print()
print(batch.keys())

### GPU Memory Check

In [ ]:
import torch
size = torch.cuda.mem_get_info()
test = list(map(lambda x:round(x/(1024*1024*1024),2),size))
status = {'available':test[0], 'total':test[1]}
status

In [ ]:
# to be added

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

### All Function Declarations

#### Func to Calculate BLEURT Score

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Elron/bleurt-base-512")
bleurt_model = AutoModelForSequenceClassification.from_pretrained("Elron/bleurt-base-512")to(device, torch.float16)
bleurt_model.eval()


In [ ]:
# prediction and groundTruth variable are expected to contain strings.

def bleurt_score(prediction, groundTruth):
    references = prediction
    candidates = groundTruth

    with torch.no_grad():
      scores = bleurt_model(**tokenizer(references, candidates, return_tensors='pt'))[0].squeeze()

    return(scores) # tensor([1.0327, 0.2055])


#### Func to Evaluate Val Loss

In [ ]:
def evaluate_validation_loss_on_val_set(small_val_set):
    
    groundTruthList = small_val_set['answers']
    predictedList = []
    bleurt_scores = []
    
    # set the model to inference mode
    
    model.eval()
    
    for i in range(len(small_val_set)):
        
        question = small_val_set[i]['questions']
        image = small_val_set[i]['images']
                
        inputs = processor(image, question, return_tensors="pt").to(device, torch.float16)
        outputs = model.generate(**inputs)
        ans = processor.decode(outputs[0], skip_special_tokens=True)
        
        # print(f'{question}? -> {ans}')

        predictedList.append(ans)
        
    for i in range(len(predictedList)):
        
        scores = bleurt_score(predictedList[i], groundTruthList[i])
        bleurt_scores.append(scores)
    
    model.train()
    
    return (sum(bleurt_scores)/len(bleurt_scores))

In [ ]:
# it means 12,32,870 records has been grouped in batch of size 16.
# so total number of batch formed is 1232870 / 16 = 77055
# that means in each epoch, total the loop will repeat for 77,055 times

tot_number_of_steps = len(train_dataloader)
number_of_epoch = 10

tot_number_of_steps,number_of_epoch

### Training Loop

In [ ]:
running_loss = 0.0
gradient_accumulation_steps = 5
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5)
model.train()

for epoch in tqdm(range(number_of_epoch)):

# Total 12,00,000 data points are there, keeping a batch size of 16 would make total 12,00,000/16 = 75000 batches
# So the below loop will repeat for 75000 times.

  for idx, batch in enumerate(train_dataloader):
    
    input_ids = batch.pop("input_ids").to(device, torch.float16)
    pixel_values = batch.pop("pixel_values").to(device, torch.float16)
    labels = batch.pop("labels").to(device, torch.float16)
    attention_mask = batch.pop('attention_mask').to(device, torch.float16)
    
    outputs = model(
                    input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=labels,
                    attention_mask = attention_mask
                   )
    
    loss = outputs.loss

    # print("Loss:", loss.item())
    print(idx+1 ,end = ' ')
    running_loss = running_loss + loss.item()
    
    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    
    # tensorboard display
    # here (epoch * tot_number_of_steps + i) denotes the current step out of the total (77054 * num_of_epoch) steps.
    # And, running_loss/100 is the mean loss of 100 iterations.
    
    if (idx+1) % 100 == 0:
        
        print(f'\n Epoch:{epoch+1}, Step:{(idx+1)/tot_number_of_steps}, Output Loss:{loss.item()} \n')
        
        val_loss = evaluate_validation_loss_on_val_set(small_val_set)
        writer.add_scalar('Training Loss', running_loss/100, epoch * tot_number_of_steps + idx)
        writer.add_scalar('Validation Loss', val_loss, epoch * tot_number_of_steps + idx)
        
        running_loss = 0
    
    # save model at last few epoch
    
    if epoch in (8,9,10):
        save_path = os.path.join("./model_chkpts/", "mod" + str(epoch))
        model.save_pretrained(save_path)

writer.close()
sys.exit()

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

For BLEURT code docs

#### Ref: https://huggingface.co/Elron/bleurt-base-512?text=I+like+you.+I+love+you

For setting up tensorboard and validation loss code position sample

#### Ref: https://www.youtube.com/watch?v=VJW9wU-1n18&ab_channel=PatrickLoeber